# Getting started with Word2Vec in Gensim and making it work!

The idea behind Word2Vec is pretty simple. We are making and assumption that you can tell the meaning of a word by the company it keeps. This is analogous to the saying *show me your friends, and I'll tell who you are*. So if you have two words that have very similar neighbors (i.e. the usage context is about the same), then these words are probably quite similar in meaning or are at least highly related. For example, the words `shocked`,`appalled` and `astonished` are typically used in a similar context. 

In this tutorial, you will learn how to use the Gensim implementation of Word2Vec and actually get it to work! I have heard a lot of complaints about poor performance etc, but its really a combination of two things, (1) your input data and (2) your parameter settings. Note that the training algorithms in this package were ported from the [original Word2Vec implementation by Google](https://arxiv.org/pdf/1301.3781.pdf) and extended with additional functionality.

### Imports and logging

First, we start with our imports and get logging established:

In [11]:
# imports needed and set up logging
import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


### Dataset 
Next, is our dataset. The secret to getting Word2Vec really working for you is to have lots and lots of text data. In this case I am going to use data from the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset. This dataset has full user reviews of cars and hotels. I have specifically concatenated all of the hotel reviews into one big file which is about 97MB compressed and 229MB uncompressed. We will use the compressed file for this tutorial. Each line in this file represents a hotel review. You can download the OpinRank Word2Vec dataset here.

To avoid confusion, while gensim’s word2vec tutorial says that you need to pass it a sequence of sentences as its input, you can always pass it a whole review as a sentence (i.e. a much larger size of text), and it should not make much of a difference. 

Now, let's take a closer look at this data below by printing the first line. You can see that this is a pretty hefty review.

In [12]:
data_file="reviews_data.txt.gz"

with gzip.open ('reviews_data.txt.gz', 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break


b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

In [13]:

def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (data_file))
logging.info ("Done reading data file")    

2020-08-15 21:23:11,547 : INFO : reading file reviews_data.txt.gz...this may take a while
2020-08-15 21:23:11,550 : INFO : read 0 reviews
2020-08-15 21:23:13,318 : INFO : read 10000 reviews
2020-08-15 21:23:15,165 : INFO : read 20000 reviews
2020-08-15 21:23:17,176 : INFO : read 30000 reviews
2020-08-15 21:23:19,086 : INFO : read 40000 reviews
2020-08-15 21:23:21,393 : INFO : read 50000 reviews
2020-08-15 21:23:23,543 : INFO : read 60000 reviews
2020-08-15 21:23:25,437 : INFO : read 70000 reviews
2020-08-15 21:23:27,034 : INFO : read 80000 reviews
2020-08-15 21:23:29,660 : INFO : read 90000 reviews
2020-08-15 21:23:31,215 : INFO : read 100000 reviews
2020-08-15 21:23:32,789 : INFO : read 110000 reviews
2020-08-15 21:23:34,460 : INFO : read 120000 reviews
2020-08-15 21:23:36,094 : INFO : read 130000 reviews
2020-08-15 21:23:37,864 : INFO : read 140000 reviews
2020-08-15 21:23:39,445 : INFO : read 150000 reviews
2020-08-15 21:23:41,295 : INFO : read 160000 reviews
2020-08-15 21:23:42,907

## Training the Word2Vec model

Training the model is fairly straightforward. You just instantiate Word2Vec and pass the reviews that we read in the previous step (the `documents`). So, we are essentially passing on a list of lists. Where each list within the main list contains a set of tokens from a user review. Word2Vec uses all these tokens to internally create a vocabulary. And by vocabulary, I mean a set of unique words.

After building the vocabulary, we just need to call `train(...)` to start training the Word2Vec model. Training on the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset takes about 10 minutes so please be patient while running your code on this dataset.

Behind the scenes we are actually training a simple neural network with a single hidden layer. But, we are actually not going to use the neural network after training. Instead, the goal is to learn the weights of the hidden layer. These weights are essentially the word vectors that we’re trying to learn. 

### Read files into a list
Now that we've had a sneak peak of our dataset, we can read it into a list so that we can pass this on to the Word2Vec model. Notice in the code below, that I am directly reading the 
compressed file. I'm also doing a mild pre-processing of the reviews using `gensim.utils.simple_preprocess (line)`. This does some basic pre-processing such as tokenization, lowercasing, etc and returns back a list of tokens (words). Documentation of this pre-processing method can be found on the official [Gensim documentation site](https://radimrehurek.com/gensim/utils.html). 



In [14]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

2020-08-15 21:24:10,465 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2020-08-15 21:24:10,467 : INFO : collecting all words and their counts
2020-08-15 21:24:10,468 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-08-15 21:24:10,758 : INFO : PROGRESS: at sentence #10000, processed 1655714 words, keeping 25777 word types
2020-08-15 21:24:11,033 : INFO : PROGRESS: at sentence #20000, processed 3317863 words, keeping 35016 word types
2020-08-15 21:24:11,341 : INFO : PROGRESS: at sentence #30000, processed 5264072 words, keeping 47518 word types
2020-08-15 21:24:11,630 : INFO : PROGRESS: at sentence #40000, processed 7081746 words, keeping 56675 word types
2020-08-15 21:24:11,964 : INFO : PROGRESS: at sentence #50000, processed 9089491 words, keeping 63744 word types
2020-08-15 21:24:12,309 : INFO : PROGRESS: at sentence #60000, processed 11013726 words, keeping 76786 word types
2020-08-15 21:24:12,560 : INFO : PROGRESS: a

2020-08-15 21:24:56,561 : INFO : EPOCH 2 - PROGRESS: at 31.54% examples, 1284144 words/s, in_qsize 20, out_qsize 1
2020-08-15 21:24:57,575 : INFO : EPOCH 2 - PROGRESS: at 34.62% examples, 1240188 words/s, in_qsize 17, out_qsize 2
2020-08-15 21:24:58,586 : INFO : EPOCH 2 - PROGRESS: at 37.70% examples, 1203941 words/s, in_qsize 20, out_qsize 2
2020-08-15 21:24:59,589 : INFO : EPOCH 2 - PROGRESS: at 40.89% examples, 1175662 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:25:00,599 : INFO : EPOCH 2 - PROGRESS: at 46.36% examples, 1200774 words/s, in_qsize 18, out_qsize 1
2020-08-15 21:25:01,610 : INFO : EPOCH 2 - PROGRESS: at 51.77% examples, 1227875 words/s, in_qsize 20, out_qsize 0
2020-08-15 21:25:02,615 : INFO : EPOCH 2 - PROGRESS: at 57.05% examples, 1250893 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:25:03,626 : INFO : EPOCH 2 - PROGRESS: at 62.34% examples, 1269116 words/s, in_qsize 17, out_qsize 2
2020-08-15 21:25:04,643 : INFO : EPOCH 2 - PROGRESS: at 65.35% examples, 1240094

2020-08-15 21:25:52,225 : INFO : EPOCH 4 - PROGRESS: at 43.65% examples, 1139186 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:25:53,232 : INFO : EPOCH 4 - PROGRESS: at 49.00% examples, 1166586 words/s, in_qsize 18, out_qsize 1
2020-08-15 21:25:54,260 : INFO : EPOCH 4 - PROGRESS: at 53.45% examples, 1176895 words/s, in_qsize 20, out_qsize 1
2020-08-15 21:25:55,263 : INFO : EPOCH 4 - PROGRESS: at 56.44% examples, 1154707 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:25:56,275 : INFO : EPOCH 4 - PROGRESS: at 59.52% examples, 1137453 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:25:57,289 : INFO : EPOCH 4 - PROGRESS: at 62.58% examples, 1122211 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:25:58,289 : INFO : EPOCH 4 - PROGRESS: at 65.73% examples, 1109183 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:25:59,304 : INFO : EPOCH 4 - PROGRESS: at 70.83% examples, 1131016 words/s, in_qsize 18, out_qsize 1
2020-08-15 21:26:00,309 : INFO : EPOCH 4 - PROGRESS: at 76.03% examples, 1151551

2020-08-15 21:26:43,289 : INFO : EPOCH 1 - PROGRESS: at 33.25% examples, 1077877 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:26:44,300 : INFO : EPOCH 1 - PROGRESS: at 38.29% examples, 1111103 words/s, in_qsize 18, out_qsize 1
2020-08-15 21:26:45,310 : INFO : EPOCH 1 - PROGRESS: at 43.78% examples, 1144085 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:26:46,322 : INFO : EPOCH 1 - PROGRESS: at 48.47% examples, 1156983 words/s, in_qsize 20, out_qsize 2
2020-08-15 21:26:47,326 : INFO : EPOCH 1 - PROGRESS: at 51.56% examples, 1137259 words/s, in_qsize 17, out_qsize 2
2020-08-15 21:26:48,350 : INFO : EPOCH 1 - PROGRESS: at 53.77% examples, 1106487 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:26:49,362 : INFO : EPOCH 1 - PROGRESS: at 56.34% examples, 1082024 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:26:50,371 : INFO : EPOCH 1 - PROGRESS: at 58.57% examples, 1056235 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:26:51,372 : INFO : EPOCH 1 - PROGRESS: at 62.88% examples, 1066545

2020-08-15 21:27:38,928 : INFO : EPOCH 3 - PROGRESS: at 37.64% examples, 1205051 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:27:39,943 : INFO : EPOCH 3 - PROGRESS: at 43.24% examples, 1233676 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:27:40,963 : INFO : EPOCH 3 - PROGRESS: at 46.46% examples, 1203692 words/s, in_qsize 18, out_qsize 1
2020-08-15 21:27:41,964 : INFO : EPOCH 3 - PROGRESS: at 49.57% examples, 1179865 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:27:42,982 : INFO : EPOCH 3 - PROGRESS: at 52.52% examples, 1156348 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:27:44,004 : INFO : EPOCH 3 - PROGRESS: at 55.63% examples, 1138892 words/s, in_qsize 18, out_qsize 1
2020-08-15 21:27:45,009 : INFO : EPOCH 3 - PROGRESS: at 59.55% examples, 1138347 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:27:46,018 : INFO : EPOCH 3 - PROGRESS: at 65.12% examples, 1162832 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:27:47,023 : INFO : EPOCH 3 - PROGRESS: at 70.07% examples, 1181888

2020-08-15 21:28:34,763 : INFO : EPOCH 5 - PROGRESS: at 39.60% examples, 1051216 words/s, in_qsize 18, out_qsize 1
2020-08-15 21:28:35,794 : INFO : EPOCH 5 - PROGRESS: at 42.99% examples, 1039325 words/s, in_qsize 16, out_qsize 3
2020-08-15 21:28:36,799 : INFO : EPOCH 5 - PROGRESS: at 46.01% examples, 1023076 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:28:37,808 : INFO : EPOCH 5 - PROGRESS: at 48.76% examples, 1008237 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:28:38,833 : INFO : EPOCH 5 - PROGRESS: at 51.72% examples, 997179 words/s, in_qsize 20, out_qsize 0
2020-08-15 21:28:39,852 : INFO : EPOCH 5 - PROGRESS: at 54.20% examples, 982852 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:28:40,870 : INFO : EPOCH 5 - PROGRESS: at 59.14% examples, 1005407 words/s, in_qsize 17, out_qsize 2
2020-08-15 21:28:41,870 : INFO : EPOCH 5 - PROGRESS: at 64.45% examples, 1030307 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:28:42,886 : INFO : EPOCH 5 - PROGRESS: at 69.07% examples, 1047423 w

2020-08-15 21:29:30,869 : INFO : EPOCH 7 - PROGRESS: at 35.06% examples, 1133673 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:29:31,891 : INFO : EPOCH 7 - PROGRESS: at 38.64% examples, 1119406 words/s, in_qsize 18, out_qsize 1
2020-08-15 21:29:32,917 : INFO : EPOCH 7 - PROGRESS: at 41.82% examples, 1096099 words/s, in_qsize 17, out_qsize 2
2020-08-15 21:29:33,923 : INFO : EPOCH 7 - PROGRESS: at 44.70% examples, 1073003 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:29:34,923 : INFO : EPOCH 7 - PROGRESS: at 47.39% examples, 1052489 words/s, in_qsize 18, out_qsize 1
2020-08-15 21:29:35,924 : INFO : EPOCH 7 - PROGRESS: at 50.29% examples, 1037184 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:29:36,933 : INFO : EPOCH 7 - PROGRESS: at 54.30% examples, 1047727 words/s, in_qsize 20, out_qsize 3
2020-08-15 21:29:37,934 : INFO : EPOCH 7 - PROGRESS: at 59.62% examples, 1075135 words/s, in_qsize 18, out_qsize 1
2020-08-15 21:29:38,937 : INFO : EPOCH 7 - PROGRESS: at 64.87% examples, 1096195

2020-08-15 21:30:26,523 : INFO : EPOCH 9 - PROGRESS: at 28.34% examples, 1179195 words/s, in_qsize 17, out_qsize 2
2020-08-15 21:30:27,532 : INFO : EPOCH 9 - PROGRESS: at 32.59% examples, 1178207 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:30:28,535 : INFO : EPOCH 9 - PROGRESS: at 36.55% examples, 1174962 words/s, in_qsize 18, out_qsize 1
2020-08-15 21:30:29,538 : INFO : EPOCH 9 - PROGRESS: at 40.71% examples, 1175168 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:30:30,556 : INFO : EPOCH 9 - PROGRESS: at 45.09% examples, 1174035 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:30:31,560 : INFO : EPOCH 9 - PROGRESS: at 49.29% examples, 1175817 words/s, in_qsize 19, out_qsize 0
2020-08-15 21:30:32,578 : INFO : EPOCH 9 - PROGRESS: at 53.07% examples, 1172585 words/s, in_qsize 17, out_qsize 2
2020-08-15 21:30:33,587 : INFO : EPOCH 9 - PROGRESS: at 57.19% examples, 1171786 words/s, in_qsize 17, out_qsize 2
2020-08-15 21:30:34,589 : INFO : EPOCH 9 - PROGRESS: at 61.15% examples, 1170272

(303495716, 415193580)

## Now, let's look at some output 
This first example shows a simple case of looking up words similar to the word `dirty`. All we need to do here is to call the `most_similar` function and provide the word `dirty` as the positive example. This returns the top 10 similar words. 

In [15]:

w1 = "dirty"
model.wv.most_similar (positive=w1)


2020-08-15 22:23:57,247 : INFO : precomputing L2-norms of word weight vectors


[('filthy', 0.8629374504089355),
 ('unclean', 0.7930690050125122),
 ('stained', 0.7691051959991455),
 ('smelly', 0.749265193939209),
 ('grubby', 0.7447434663772583),
 ('dusty', 0.7446684837341309),
 ('dingy', 0.725252091884613),
 ('disgusting', 0.7216766476631165),
 ('gross', 0.7210666537284851),
 ('soiled', 0.7109328508377075)]

In [23]:
#this is a temporary cell added by me, Chiu
#w1 = "man"
#model.wv.most_similar (w1)
model.most_similar("european")

<ipython-input-23-54b7374fe453>:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar("european")


[('eurpoean', 0.6736182570457458),
 ('american', 0.6519259214401245),
 ('europe', 0.6427686214447021),
 ('boutique', 0.5354989767074585),
 ('europeon', 0.5097432136535645),
 ('botique', 0.5068238973617554),
 ('arabic', 0.4953358471393585),
 ('traditional', 0.48308032751083374),
 ('europeans', 0.4760194718837738),
 ('europen', 0.4705777168273926)]

That looks pretty good, right? Let's look at a few more. Let's look at similarity for `polite`, `france` and `shocked`. 

In [7]:
# look up top 6 words similar to 'polite'
w1 = ["polite"]
model.wv.most_similar (positive=w1,topn=6)


NameError: name 'model' is not defined

In [8]:
# look up top 6 words similar to 'france'
w1 = ["france"]
model.wv.most_similar (positive=w1,topn=6)


NameError: name 'model' is not defined

In [9]:
# look up top 6 words similar to 'shocked'
w1 = ["shocked"]
model.wv.most_similar (positive=w1,topn=6)


NameError: name 'model' is not defined

That's, nice. You can even specify several positive examples to get things that are related in the provided context and provide negative examples to say what should not be considered as related. In the example below we are asking for all items that *relate to bed* only:

In [10]:
# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)


NameError: name 'model' is not defined

### Similarity between two words in the vocabulary

You can even use the Word2Vec model to return the similarity between two words that are present in the vocabulary. 

In [12]:
# similarity between two different words
model.wv.similarity(w1="dirty",w2="smelly")

0.76058316

In [13]:
# similarity between two identical words
model.wv.similarity(w1="dirty",w2="dirty")

1.0

In [14]:
# similarity between two unrelated words
model.wv.similarity(w1="dirty",w2="clean")

0.27513152

Under the hood, the above three snippets computes the cosine similarity between the two specified words using word vectors of each. From the scores, it makes sense that `dirty` is highly similar to `smelly` but `dirty` is dissimilar to `clean`. If you do a similarity between two identical words, the score will be 1.0 as the range of the cosine similarity score will always be between [0.0-1.0]. You can read more about cosine similarity scoring [here](https://en.wikipedia.org/wiki/Cosine_similarity).

### Find the odd one out
You can even use Word2Vec to find odd items given a list of items.

In [15]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["cat","dog","france"])

C:\Users\Eric\anaconda3\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'france'

In [16]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["bed","pillow","duvet","shower"])


'shower'

## Understanding some of the parameters
To train the model earlier, we had to set some parameters. Now, let's try to understand what some of them mean. For reference, this is the command that we used to train the model.

```
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
```

### `size`
The size of the dense vector to represent each token or word. If you have very limited data, then size should be a much smaller value. If you have lots of data, its good to experiment with various sizes. A value of 100-150 has worked well for me. 

### `window`
The maximum distance between the target word and its neighboring word. If your neighbor's position is greater than the maximum window width to the left and the right, then, some neighbors are not considered as being related to the target word. In theory, a smaller window should give you terms that are more related. If you have lots of data, then the window size should not matter too much, as long as its a decent sized window. 

### `min_count`
Minimium frequency count of words. The model would ignore words that do not statisfy the `min_count`. Extremely infrequent words are usually unimportant, so its best to get rid of those. Unless your dataset is really tiny, this does not really affect the model.

### `workers`
How many threads to use behind the scenes?


## When should you use Word2Vec?

There are many application scenarios for Word2Vec. Imagine if you need to build a sentiment lexicon. Training a Word2Vec model on large amounts of user reviews helps you achieve that. You have a lexicon for not just sentiment, but for most words in the vocabulary. 

Beyond, raw unstructured text data, you could also use Word2Vec for more structured data. For example, if you had tags for a million stackoverflow questions and answers, you could find tags that are related to a given tag and recommend the related ones for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. Granted, you still need a large number of examples to make it work. 
